In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import cv2
import random
import pickle
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD

Training data unzip after unzip training image folder is **/kaggle/working/train**

In [ ]:
with ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip', 'r') as zip:
    zip.extractall()
    print('done')

Testing data unzip after testing image folder is **/kaggle/working/testing**

In [ ]:
with ZipFile('/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip', 'r') as zip:
    zip.extractall()
    print('done')

In [ ]:
PATH = '/kaggle/working/train'
filename = os.listdir(PATH)
IMG_SIZE = 100

plt.figure(figsize=(10,10))
for i in range(1, 7):
    img_array = cv2.imread(os.path.join(PATH, filename[i]))
    resize_image = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    plt.subplot(3,3, i)
    image = cv2.cvtColor(resize_image, cv2.COLOR_BGR2RGB)
    #plt.figure()
    plt.axis('off')
    plt.imshow(image)    

In [ ]:
training_data = []
x_train=  []
y_train = []

for files in tqdm(os.listdir(PATH)):
    try:
        
        if files.find('cat') == -1:
            category = 0
        else:
            category = 1   
        img_array = cv2.imread(os.path.join(PATH, files), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        training_data.append([new_array, category])
        x_train.append([new_array])
        y_train.append([category])
    except Exception as e:
        pass



In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
type(training_data)

In [ ]:
x_train = x_train.reshape(25000, 100, 100, 1)

In [ ]:
x_train.shape

In [ ]:
testing_data = []
PATH = '/kaggle/working/test'

for files in tqdm(os.listdir(PATH)):
    try:
        img_array = cv2.imread(os.path.join(PATH, files), cv2.IMREAD_GRAYSCALE)
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        testing_data.append([new_array])
    except Exception as e:
        pass

In [ ]:
plt.figure(figsize=(10,10))
for i in range(1, 7):
    plt.subplot(3,3,i)
    plt.axis('off')
    if training_data[10+i][1] == 0:
        plt.title('Dog')
    else:
        plt.title('Cat')
    plt.imshow(training_data[10+i][0], cmap='gray_r')

In [ ]:
random.seed(45)
random.shuffle(training_data)

In [ ]:
print('length of training data', str(len(training_data)))
print('Shape of image', training_data[0][0].shape)

Total training data image is 25000 and one image size is 100x100. But we don't want use complete image. 

In [ ]:
X = []
y = []

for features, labels in training_data:
    X.append(features.reshape(100,100, 1))
    y.append(labels)



In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train,y_train, test_size = 0.3, random_state=51)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train[0]

In [ ]:
# create model
model = Sequential()

model.add(Conv2D(32, kernel_size = (5,5), activation='relu', input_shape = (100,100, 1)))

model.add(Conv2D(64, kernel_size = (5,5), activation='relu'))

model.add(MaxPooling2D(pool_size=(4,4)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(2, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=SGD(0.001), metrics =['accuracy'])

In [ ]:
model.summary()

In [ ]:
y_test.shape

In [ ]:
batch_size = 32
epochs = 10

history = model.fit(X_train, y_train, batch_size=batch_size, epochs = epochs, verbose=1,
                   validation_data = (X_test, y_test))

score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[0])

In [ ]:
# Plotting our loss charts
# Use the history object we created to get our saved perormace result.
history_dict = history.history

# Extract the loss and validation losses
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']

epochs = range(1, len(loss_values)+1)

line1 = plt.plot(epochs, val_loss_values, label ='Validation/Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth = 2.0, marker='+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker='4', markersize=10.0)
plt.grid(True)
plt.show()

### Accuracy Charts

In [ ]:
history_dict = history.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values)+1)

line1 = plt.plot(epochs, val_loss_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label ='Training Accuracy')

plt.setp(line1, linewidth=2.0, marker='+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker='4', markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()